<a href="https://colab.research.google.com/github/ItisShreyadav/Rock_Paper_Scissors_Game/blob/main/Prediction_for_Chronic_Kidney_Disease_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
import pandas as pd
import numpy as np
from collections import Counter as c
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection  import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
import pickle


In [37]:
data=pd.read_csv("/content/kidney_disease.csv")
data.head()

,id,age,bp,sg,al,su,rbc,pc,pcc,ba,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,...,44,7800,5.2,yes,yes,no,good,no,no,ckd
1,1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,...,38,6000,NaN,no,no,no,good,no,no,ckd
2,2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,...,31,7500,NaN,no,yes,no,poor,no,yes,ckd
3,3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,...,32,6700,3.9,yes,no,no,poor,yes,yes,ckd
4,4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,...,35,7300,4.6,no,no,no,good,no,no,ckd


data.drop(["id"],axis=1,inplace=True)
***here this line means drop- id column =(axis 1)if row = axis 0,
 inplace = true (because it saves memory and chngs made directly in given dataset)***

In [38]:
data.columns

Index(['id', 'age', 'bp', 'sg', 'al', 'su', 'rbc', 'pc', 'pcc', 'ba', 'bgr',
       'bu', 'sc', 'sod', 'pot', 'hemo', 'pcv', 'wc', 'rc', 'htn', 'dm', 'cad',
       'appet', 'pe', 'ane', 'classification'],
      dtype='object')

In [39]:
data.drop(["id"],axis=1,inplace=True)

In [40]:
data.columns=[ 'age', 'blood_pressure', 'sfecific_gravity', 'albumin', 'sugar', 'red_blood_cells', 'pus_cell', 'pus_cell_clumps', 'bacteria', 'blood_glucose_random', 'blue_ureau',
       'serum_creatinine', 'sodium', 'potassium', 'hemoglobin', 'packed_cell_volume', 'white_blood_cell_count', 'red_blood_cell_count', 'hypertension', 'diabetesmellitus', 'coronary_artery_disease',
       'appetite', 'pedal_edema', 'anemia', 'class']
data.columns

Index(['age', 'blood_pressure', 'sfecific_gravity', 'albumin', 'sugar',
       'red_blood_cells', 'pus_cell', 'pus_cell_clumps', 'bacteria',
       'blood_glucose_random', 'blue_ureau', 'serum_creatinine', 'sodium',
       'potassium', 'hemoglobin', 'packed_cell_volume',
       'white_blood_cell_count', 'red_blood_cell_count', 'hypertension',
       'diabetesmellitus', 'coronary_artery_disease', 'appetite',
       'pedal_edema', 'anemia', 'class'],
      dtype='object')

In [41]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 25 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   age                      391 non-null    float64
 1   blood_pressure           388 non-null    float64
 2   sfecific_gravity         353 non-null    float64
 3   albumin                  354 non-null    float64
 4   sugar                    351 non-null    float64
 5   red_blood_cells          248 non-null    object 
 6   pus_cell                 335 non-null    object 
 7   pus_cell_clumps          396 non-null    object 
 8   bacteria                 396 non-null    object 
 9   blood_glucose_random     356 non-null    float64
 10  blue_ureau               381 non-null    float64
 11  serum_creatinine         383 non-null    float64
 12  sodium                   313 non-null    float64
 13  potassium                312 non-null    float64
 14  hemoglobin               3

In [42]:
data['class'].unique() #is used to check all the unique values in the column named "class" of a DataFrame called data.



array(['ckd', 'ckd\t', 'notckd'], dtype=object)

In [43]:
data['class']=data['class'].replace("ckd\t",'ckd') # It replaces any occurrence of "ckd\t" (which has a tab character \t at the end) with "ckd" (clean version).
data['class'].unique()

array(['ckd', 'notckd'], dtype=object)

In [44]:
catcols=set(data.dtypes[data.dtypes=='O'].index.values) # It finds and prints all columns in the dataset that contain text or categorical data

print(catcols)

{'diabetesmellitus', 'bacteria', 'packed_cell_volume', 'class', 'red_blood_cells', 'pus_cell', 'red_blood_cell_count', 'hypertension', 'appetite', 'coronary_artery_disease', 'pedal_edema', 'anemia', 'white_blood_cell_count', 'pus_cell_clumps'}


In [45]:
for i in catcols:
  print("Columns:",i)
  print(c(data[i]))
  print('*'*120+'\n')


Columns: diabetesmellitus
Counter({'no': 258, 'yes': 134, '\tno': 3, '\tyes': 2, nan: 2, ' yes': 1})
************************************************************************************************************************

Columns: bacteria
Counter({'notpresent': 374, 'present': 22, nan: 4})
************************************************************************************************************************

Columns: packed_cell_volume
Counter({nan: 70, '52': 21, '41': 21, '44': 19, '48': 19, '40': 16, '43': 14, '45': 13, '42': 13, '32': 12, '36': 12, '33': 12, '28': 12, '50': 12, '37': 11, '34': 11, '35': 9, '29': 9, '30': 9, '46': 9, '31': 8, '39': 7, '24': 7, '26': 6, '38': 5, '47': 4, '49': 4, '53': 4, '51': 4, '54': 4, '27': 3, '22': 3, '25': 3, '23': 2, '19': 2, '16': 1, '\t?': 1, '14': 1, '18': 1, '17': 1, '15': 1, '21': 1, '20': 1, '\t43': 1, '9': 1})
************************************************************************************************************************

Col

In [46]:
catcols.remove('red_blood_cell_count')
catcols.remove('packed_cell_volume')
catcols.remove('white_blood_cell_count')
print(catcols)

{'diabetesmellitus', 'bacteria', 'class', 'red_blood_cells', 'pus_cell', 'hypertension', 'appetite', 'coronary_artery_disease', 'pedal_edema', 'anemia', 'pus_cell_clumps'}


In [47]:
contcols=set(data.dtypes[data.dtypes!='O'].index.values)
print(contcols)
for i in contcols:
  print("Columns:",i)
  print(c(data[i]))
  print('*'*120+'\n')

{'age', 'sfecific_gravity', 'serum_creatinine', 'potassium', 'albumin', 'sodium', 'blood_pressure', 'sugar', 'blood_glucose_random', 'hemoglobin', 'blue_ureau'}
Columns: age
Counter({60.0: 19, 65.0: 17, 48.0: 12, 50.0: 12, 55.0: 12, 47.0: 11, 62.0: 10, 45.0: 10, 54.0: 10, 59.0: 10, 56.0: 10, 61.0: 9, 70.0: 9, 46.0: 9, 34.0: 9, 68.0: 8, 73.0: 8, 64.0: 8, 71.0: 8, 57.0: 8, 63.0: 7, 72.0: 7, 67.0: 7, 30.0: 7, 42.0: 6, 69.0: 6, 35.0: 6, 44.0: 6, 43.0: 6, 33.0: 6, 51.0: 5, 52.0: 5, 53.0: 5, 75.0: 5, 76.0: 5, 58.0: 5, 41.0: 5, 66.0: 5, 24.0: 4, 40.0: 4, 39.0: 4, 80.0: 4, 23.0: 4, 74.0: 3, 38.0: 3, 17.0: 3, 8.0: 3, 32.0: 3, 37.0: 3, 25.0: 3, 29.0: 3, 21.0: 2, 15.0: 2, 5.0: 2, 12.0: 2, 49.0: 2, 19.0: 2, 36.0: 2, 20.0: 2, 28.0: 2, 7.0: 1, nan: 1, 82.0: 1, 11.0: 1, 26.0: 1, nan: 1, nan: 1, nan: 1, nan: 1, 81.0: 1, 14.0: 1, 27.0: 1, nan: 1, 83.0: 1, 4.0: 1, 3.0: 1, 6.0: 1, nan: 1, 90.0: 1, 78.0: 1, nan: 1, 2.0: 1, nan: 1, 22.0: 1, 79.0: 1})
********************************************************

In [48]:
print(contcols)

{'age', 'sfecific_gravity', 'serum_creatinine', 'potassium', 'albumin', 'sodium', 'blood_pressure', 'sugar', 'blood_glucose_random', 'hemoglobin', 'blue_ureau'}


In [49]:
contcols.add('red_blood_cell_count')

contcols.add('white_blood_cell_count')
print(contcols)

{'age', 'sfecific_gravity', 'serum_creatinine', 'potassium', 'red_blood_cell_count', 'albumin', 'sodium', 'blood_pressure', 'sugar', 'white_blood_cell_count', 'blood_glucose_random', 'hemoglobin', 'blue_ureau'}


In [50]:
contcols.add('sfecific_gravity')
contcols.add('albumin')
contcols.add('sugar')
print(contcols)

{'age', 'sfecific_gravity', 'serum_creatinine', 'potassium', 'red_blood_cell_count', 'albumin', 'sodium', 'blood_pressure', 'sugar', 'white_blood_cell_count', 'blood_glucose_random', 'hemoglobin', 'blue_ureau'}


In [52]:
data['diabetesmellitus']=data.coronary_artery_disease.replace(to_replace={'\tno':'no','\tyes':'yes','yes':'yes'})
c(data['diabetesmellitus'])

Counter({'no': 364, 'yes': 34, nan: 2})

In [53]:
data.isnull().any()

,0
age,True
blood_pressure,True
sfecific_gravity,True
albumin,True
sugar,True
red_blood_cells,True
pus_cell,True
pus_cell_clumps,True
bacteria,True
blood_glucose_random,True


In [54]:
data.isnull().sum()

,0
age,9
blood_pressure,12
sfecific_gravity,47
albumin,46
sugar,49
red_blood_cells,152
pus_cell,65
pus_cell_clumps,4
bacteria,4
blood_glucose_random,44


In [56]:
data.packed_cell_volume=pd.to_numeric(data.packed_cell_volume,errors='coerce') # It converts the "packed_cell_volume" column values to numbers, and if any value can't be converted (like a typo or text), it replaces it with NaN (missing value).
data.white_blood_cell_count=pd.to_numeric(data.white_blood_cell_count,errors='coerce')
data.red_blood_cell_count=pd.to_numeric(data.red_blood_cell_count,errors='coerce')